In [2]:
import os
import numpy as np

import torch
import torchvision
from torch import nn
from torchvision import transforms

from PIL import Image

In [5]:
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [6]:
rootPath = "./images/"
images = os.listdir(rootPath)

# 数据预处理
transform = transforms.Compose([
    transforms.Resize((256 , 256)) ,
    transforms.ToTensor() ,
    transforms.Normalize(mean = [0.485 , 0.456 , 0.406] , std = [0.229 , 0.224 , 0.225])
])

model = torchvision.models.resnet18(weights = "DEFAULT")  # 预训练的 ResNet18
model.avgpool.register_forward_hook(get_activation("avgpool"))  # 得到特征向量
model.eval()  # 评估模式

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
fileNames = []
featureVectors = None  # np.array

with torch.no_grad():
    for i, fileName in enumerate(images):
        try:
            fileNames.append(fileName)
            
            try:
                image = Image.open(rootPath + fileName)
            except:
                print(f"Failed to open image {fileName}.")
                continue
            
            image = transform(image)
            output = model(image[None, ...])
            featureVector = activation["avgpool"].numpy().squeeze()[None, ...]
            
            if featureVectors is None:  # 第一个
                featureVectors = featureVector
            else:  # 非第一个
                featureVectors = np.vstack([featureVectors, featureVector])
                
        except:  # 读取文件出错
            continue
        
        # 每处理 100 张图片输出提示信息
        if i % 100 == 0:
            print(f"Finished processing images {0 if i == 0 else i - 100 + 1} ~ {i}.")

# 保存成 NumPy 文件
print("Processing finished.")
np.save("fileNames.npy" , fileNames)
np.save("featureVectors.npy" , featureVectors)

Finished processing images 0 ~ 0.
Finished processing images 1 ~ 100.
Finished processing images 101 ~ 200.
Finished processing images 201 ~ 300.
Finished processing images 301 ~ 400.
Finished processing images 401 ~ 500.
Finished processing images 501 ~ 600.
Finished processing images 601 ~ 700.
Finished processing images 701 ~ 800.
Finished processing images 801 ~ 900.
Finished processing images 901 ~ 1000.
Finished processing images 1001 ~ 1100.
Finished processing images 1101 ~ 1200.
Finished processing images 1201 ~ 1300.
Finished processing images 1301 ~ 1400.


False